In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import datetime
% matplotlib inline

In [2]:
#path = "../../../Google Drive/Data_science/NYU/Machine Learning/ML Project (Collisions)/" #Joe
path = "../../../../Google Drive/ML Project (Collisions)/" # Joyce
# path = "" # Lucas

In [3]:
df = pkl.load(open(path + 'data_for_training/v2/predrop_collisions.pkl','rb'))

In [4]:
train_df = df[pd.to_datetime(df.date_time).dt.date <= datetime.date(2015,9,11)]
val_df = df[(pd.to_datetime(df.date_time).dt.date >= datetime.date(2015,9,12)) & (pd.to_datetime(df.date_time).dt.date <= datetime.date(2016,7,30))]
test_df = df[pd.to_datetime(df.date_time).dt.date >= datetime.date(2016,7,31)]

In [5]:
index_df = df[['longitude','latitude','day_of_week','hour_of_day']].dropna()

steps = [.01,.03,.06,.1,.3,.6,1,10]
train_df_avg = train_df.copy()
for step in steps:
    
    # create bins
    to_bin = lambda x: np.round(x / step) * step
    for df in [train_df_avg, index_df]:
        df['Lat_bin'] = df.latitude.map(to_bin)
        df['Long_bin'] = df.longitude.map(to_bin)
    
    #groupby
    avg_df = train_df_avg.groupby(['Lat_bin', 'Long_bin','hour_of_day','day_of_week'])['injured_or_killed'].mean()
    
    # Add step to name
    if isinstance(avg_df,pd.Series):
        avg_df.name = avg_df.name + ' - ' + str(step)
    else:
        avg_df.columns = map(lambda x: x + ' - ' + str(step),avg_df.columns)
        
    index_df = index_df.join(avg_df, on = avg_df.index.names).fillna(0)
    index_df.drop(['Lat_bin','Long_bin'],axis=1, inplace=True)

    print ("Finished step: %s" % (step))
index_df.drop(['longitude','latitude','day_of_week','hour_of_day'],axis=1, inplace=True)
index_df.index.name = 'UNIQUE KEY'

Finished step: 0.01
Finished step: 0.03
Finished step: 0.06
Finished step: 0.1
Finished step: 0.3
Finished step: 0.6
Finished step: 1
Finished step: 10


In [6]:
pkl.dump(index_df, open(path+'data_for_joining/injury_avgs.pkl','wb'))

In [ ]:
val_joined = val_df.join(index_df, on='unique_key')

## Avg. Baslines

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
for step in steps:
    print(roc_auc_score(val_joined.injured_or_killed,val_joined['injured_or_killed - '+str(step)]))

## More Baslines

In [ ]:
df = pkl.load(open(path + 'collisions_1hot.pkl','rb'))

In [ ]:
df.date_time_proper = df.date_time.apply(lambda x: datetime.datetime.fromtimestamp(int(x)/1e9))

In [ ]:
train_df = df[df.date_time_proper.dt.date <= datetime.date(2015,9,11)]
val_df = df[(df.date_time_proper.dt.date >= datetime.date(2015,9,12)) & (df.date_time_proper.dt.date <= datetime.date(2016,7,30))]
# test_df = df[df.date_time.dt.date >= datetime.date(2016,7,31)]

In [ ]:
datetime.datetime.fromtimestamp(int(df.date_time.iloc[0]/1e9))

In [ ]:
val_df['global_mean'] = train_df.injured_or_killed.mean()

In [ ]:
roc_auc_score(val_df.injured_or_killed,val_df['global_mean'])